In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
import pandas as pd
import numpy as np
import os,cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
train=pd.read_csv('train.csv')
train.head()

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


In [ ]:
base='Dance'
train_dir = os.path.join(str(base)+ '/train/')
test_dir = os.path.join(str(base)+'/test/')

train_fnames = os.listdir(train_dir)
test_fnames = os.listdir(test_dir)

print(train_fnames[:9])
print(test_fnames[:9])
print(train.Image)

['409.jpg', '1.jpg', '504.jpg', '289.jpg', '87.jpg', '46.jpg', '69.jpg', '353.jpg', '5.jpg']
['232.jpg', '429.jpg', '411.jpg', '20.jpg', '175.jpg', '196.jpg', '508.jpg', '114.jpg', '506.jpg']


In [ ]:
img_width = 224
img_height = 224

In [ ]:

# this function reads image from the disk,train file for image and class maping and returning output in numpy array formate
# for input and target data
def train_data_preparation(list_of_images, train, train_dir):
    train_data = [] 
    train_label = [] 
    for image in list_of_images:
        train_data.append(cv2.resize(cv2.imread(train_dir+image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))
        if image in list(train['Image']):
            train_label.append(train.loc[train['Image'] == image, 'target'].values[0])
    
            
    return train_data, train_label

def test_data_prepare(list_of_images, test_dir):
    test_data = [] 
    for image in list_of_images:
        test_data.append(cv2.resize(cv2.imread(test_dir+image), (img_width,img_height), interpolation=cv2.INTER_CUBIC)) 
            
    return test_data


In [ ]:
training_data, training_labels = train_data_preparation(train_fnames, train, train_dir)
testing_data = test_data_prepare(test_fnames,test_dir)

In [ ]:
print(train.loc[train['Image'] == '163.jpg','target'])

1    bharatanatyam
Name: target, dtype: object


In [ ]:
le =LabelEncoder()
training_labels=le.fit_transform(training_labels)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(training_data, training_labels, test_size=0.33, random_state=42)

In [ ]:
train_datagenerator = ImageDataGenerator(
        rescale=1. / 255,
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        rotation_range=40,  
        zoom_range = 0.2,  
        shear_range=0.2,
        width_shift_range=0.10,  
        height_shift_range=0.10,  
        horizontal_flip=True,  
        vertical_flip=False) 


val_datagenerator=ImageDataGenerator(
        rescale=1. / 255
)

train_datagenerator.fit(X_train)
val_datagenerator.fit(X_val)
X_train=np.array(X_train)
X_val=np.array(X_val)

In [ ]:
vggmodel =VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3),pooling='max')

 # Print the model summary
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:

vggmodel.trainable = False
model = tf.keras.models.Sequential([
                                    vggmodel,
                                    tf.keras.layers.Dense(1024,activation='relu'),
                                    tf.keras.layers.Dropout(0.15),
                                    tf.keras.layers.Dense(256,activation='relu'),
                                    tf.keras.layers.Dropout(0.15),
                                    tf.keras.layers.Dense(8,activation='softmax'),
])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 2056      
Total params: 15,504,456
Trainable params: 789,768
Non-trainable params: 14,714,688
____________________________________

In [ ]:
reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history =model.fit_generator(
    train_datagenerator.flow(X_train, to_categorical(y_train,8), batch_size=16),
    validation_data=val_datagenerator.flow(X_val, to_categorical(y_val,8), batch_size=16),
    verbose=2,
    epochs=30,
    callbacks=callbacks
)

Epoch 1/30
16/16 - 5s - loss: 0.5417 - accuracy: 0.8107 - val_loss: 1.2198 - val_accuracy: 0.6777 - lr: 0.0010
Epoch 2/30
16/16 - 5s - loss: 0.5974 - accuracy: 0.7942 - val_loss: 1.6197 - val_accuracy: 0.5785 - lr: 0.0010
Epoch 3/30

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
16/16 - 5s - loss: 0.5575 - accuracy: 0.8148 - val_loss: 1.4751 - val_accuracy: 0.5702 - lr: 0.0010
Epoch 4/30
16/16 - 5s - loss: 0.2959 - accuracy: 0.8889 - val_loss: 1.2961 - val_accuracy: 0.6446 - lr: 1.0000e-04
Epoch 5/30
16/16 - 4s - loss: 0.2220 - accuracy: 0.9383 - val_loss: 1.2836 - val_accuracy: 0.6198 - lr: 1.0000e-04
Epoch 6/30
16/16 - 4s - loss: 0.2020 - accuracy: 0.9383 - val_loss: 1.3253 - val_accuracy: 0.6116 - lr: 1.0000e-04
Epoch 7/30
16/16 - 5s - loss: 0.1956 - accuracy: 0.9424 - val_loss: 1.3432 - val_accuracy: 0.6116 - lr: 1.0000e-04
Epoch 8/30
16/16 - 5s - loss: 0.1828 - accuracy: 0.9465 - val_loss: 1.3739 - val_accuracy: 0.6033 - lr: 1.0000e-04
Epoch 9/30